In [169]:
import numpy as np
from fooof import FOOOF
from importlib import reload
import os
import mne
from matplotlib import pyplot as plt
# import own functions
from utils import find_folders
import fooof_ftg_fun

### Add Directories/ Load Files

In [170]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [218]:
fft_np = np.load(os.path.join(
    project_path,
        'data',
        'FFTs',
        'with_med_FTG',
        'Sub045_FFT.npy'
    )
)

SUBID = 'SUB_045'
ON = 52
OFF = 82
SIDE = 0

spectrum = np.mean(fft_np[SIDE,:,ON:OFF],1)

In [219]:
plt.plot(spectrum)

In [220]:
reload(fooof_ftg_fun)
PATH = os.path.join(project_path, 'results', 'FOOOF_FIT')
fm = fooof_ftg_fun.fit_fooof(SUBID, PATH, spectrum)

fm.print_results()


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 1.00 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

                                                                                                  
                                   FOOOF - POWER SPECTRUM MODEL                                   
                                                                                                  
                       The model was run on the frequency range 60 - 90 Hz                        
                                 Frequency Resolution is 1.00 Hz                                  
                                                                                                  
                            Aperiodic Parameters (offs